# 目录
## 1. 导入模块
## 2. 导入加利福尼亚房价数据，并写入CSV
## 3. dataset读取CSV文件数据
### 3.1 tf.data.Dataset.list_files 分装文件名列表
  - `tf.data.Dataset.list_files`
  - `tf.data.Dataset.from_tensor_slices`
  
### 3.2 interleave (tf.data.TextLineDataset) 读取文件列表的数据
  - `interleave`
  - `tf.data.TextLineDataset` 和 `skip`

### 3.3 tf.io.decode_csv解析字符串数据
  - `tf.io.decode_csv`

### 3.4 合起来封装成一个函数
  - `tf.data.Dataset.list_files`
  - `tf.data.Dataset.from_tensor_slices`
  - `interleave`
  - `tf.data.TextLineDataset` 和 `skip`
  - `tf.io.decode_csv`
  - `tf.stack`
  - `map`
  - `shuffle`
  - `repeat`
  - `batch`

### 3.5 生成train valid test 的dataset
## 4. 使用dataset 训练、测试

## 1. 导入模块

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

from tensorflow import keras
import tensorflow as tf
import sys
import os
import time
import datetime

for module in [np, pd, mpl, sklearn, keras, tf]:
    print(module.__name__, module.__version__)

numpy 1.18.1
pandas 0.25.3
matplotlib 3.1.2
sklearn 0.22.1
tensorflow_core.python.keras.api._v2.keras 2.2.4-tf
tensorflow 2.1.0


## 2. 导入加利福尼亚房价数据，并写入CSV

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()

x_train_all, x_test, y_train_all, y_test = train_test_split(housing.data, housing.target, random_state=1234)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_all, y_train_all, random_state=1234)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [3]:
output_dir = "generate_csv"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
def save_to_csv(output_dir, data, name_prefix, headers=None, n_parts=10):
    path_format = os.path.join(output_dir, "{}_{:02d}.csv")
    file_names = []
    
    for file_index, row_indices in enumerate(np.array_split(np.arange(len(data)), n_parts)):
        part_file = path_format.format(name_prefix, file_index)
        file_names.append(part_file)
        
        with open(part_file, "wt", encoding="utf-8") as f:
            if headers is not None:
                f.write(headers+"\n")
            for row_index in row_indices:
                data_str = ",".join([repr(col) for col in data[row_index]])
                f.write(data_str + "\n")
                
    return file_names
            
train_data = np.c_[x_train, y_train]
valid_data = np.c_[x_valid, y_valid]
test_data = np.c_[x_test, y_test]
header_str = ",".join(housing.feature_names + ["MidianHouseValue"])

train_filenames = save_to_csv(output_dir, train_data, "train", headers=header_str, n_parts=20)
valid_filenames = save_to_csv(output_dir, valid_data, "valid", headers=header_str, n_parts=10)
test_filenames = save_to_csv(output_dir, test_data, "test", headers=header_str, n_parts=10)


## 3. dataset读取CSV文件数据
> 1. tf.data.Dataset.list_files 分装文件名列表<br/>
2. interleave (tf.data.TextLineDataset) 读取文件列表的数据<br/>
3. tf.io.decode_csv 解析数据


### 3.1 tf.data.Dataset.list_files 分装文件名列表

In [4]:
filenames_dataset = tf.data.Dataset.list_files(train_filenames)
for filename in filenames_dataset.take(5):
    print(filename)

tf.Tensor(b'generate_csv/train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_10.csv', shape=(), dtype=string)


In [15]:
filenames_dataset = tf.data.Dataset.from_tensor_slices(train_filenames)
for filename in filenames_dataset.take(5):
    print(filename)

tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_01.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_02.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_04.csv', shape=(), dtype=string)


>目前所知， `tf.data.Dataset.from_tensor_slices` 和 `tf.data.Dataset.list_files` 的效果一样

### 3.2 interleave (tf.data.TextLineDataset) 读取文件列表的数据

In [5]:
raw_dataset = filenames_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filename).skip(1), # skip 跳过CSV文件的header
    cycle_length=5
)

for raw_item in raw_dataset.take(4):
    print(raw_item)

tf.Tensor(b'0.5902555211727188,-1.0047716525188841,0.21699232000464558,-0.12981269476922802,-0.8572437318334718,-0.0321533002773346,1.5783419299912835,-0.7683382058137005,1.923', shape=(), dtype=string)
tf.Tensor(b'1.7098603165469446,-0.28425041904670467,1.0304884304939064,-0.15087509238871963,0.1922770273226248,6.041362108085698e-05,-0.8738403492849366,0.7601019943019404,2.949', shape=(), dtype=string)
tf.Tensor(b'0.13032274828980675,-0.6845399931979155,0.08409426834023241,-0.33923810587324865,-0.23015977197991858,0.02396275063956827,-1.126546156691264,1.1596942034825073,1.625', shape=(), dtype=string)
tf.Tensor(b'1.545255545502697,-1.0047716525188841,0.6293517735071292,-0.1617293906834496,2.2256061546322004,-0.009899126784783673,-0.5790169073108862,0.505361960949337,3.233', shape=(), dtype=string)


### 3.3 tf.io.decode_csv解析字符串数据

In [6]:
# tf.io.decode_csv(str, record_defaults)
test_str = "1,2,3,4,5,6"
record_defaults = [tf.constant(0, dtype=tf.int32), 0, np.nan, "hi", tf.constant([]), 1.0]

parsed_fields = tf.io.decode_csv(test_str, record_defaults) # 按照 record_defaults 的对应的数据类型解析 test_str

print(parsed_fields)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: shape=(), dtype=string, numpy=b'4'>, <tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


### 3.4 合起来封装成一个函数

In [7]:
def csv_reader_dataset(filenames, n_readers=5, n_fields=9, n_threads=5, epochs=None, batch_size=32, shuffle=True):

    # 1. dataset 分装文件列表
    filenames_dataset = tf.data.Dataset.from_tensor_slices(filenames)
    # filenames_dataset = tf.data.Dataset.list_files(filenames)
    
    # 2. interleave 根据文件名 读取文件数据
    raw_dataset = filenames_dataset.interleave( # interleave 下面函数返回的元素，组合成一个dataset
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length=n_readers
    )
    
    # 3. 使用map 和 tf.io.decode_csv 对应解析
    def parse_csv_line(line, n_fields):
        default_fields = [tf.constant(np.nan)]* n_fields
        parsed_line = tf.io.decode_csv(line, default_fields) # 解析出来的是一个列表，列表的元素是一个0维的tensor

        # tf.stack 把 parsed_line 转化为一个向量tensor
        x = tf.stack(parsed_line[:-1])
        y = tf.stack(parsed_line[-1:])

        return x, y
    
    dataset = raw_dataset.map(  # map 不改变dataset的元素数量，只改变元素的数据结构
        map_func=lambda line: parse_csv_line(line, n_fields),
        num_parallel_calls=n_threads
    )
    
    # 4. shuffle repeat batch
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    
    return dataset

In [8]:
dataset = csv_reader_dataset(train_filenames, n_readers=5, n_fields=9, n_threads=1, epochs=1, batch_size=4, shuffle=True)
for x, y in dataset.take(1):
    print(x)
    print(y)

tf.Tensor(
[[-0.9824953   1.8773133   0.03170636  0.09444513 -0.3681558   0.01779718
  -0.76152664  0.63023454]
 [-1.1782669  -0.2842504   0.40971473  1.4811543  -1.0487484  -0.05024404
   2.4066553  -1.857227  ]
 [-1.0428973  -1.0047717  -0.46401903 -0.13675326  1.558156    0.1401103
   0.02466919  0.17070349]
 [ 1.4205359   0.276155    0.48176667 -0.13708918 -0.3644008  -0.0439072
  -0.7662064   0.5852804 ]], shape=(4, 8), dtype=float32)
tf.Tensor(
[[1.28 ]
 [0.792]
 [0.552]
 [3.692]], shape=(4, 1), dtype=float32)


### 3.5 生成train  valid  test 的dataset

In [9]:
batch_size = 32
epochs = 100

train_dataset = csv_reader_dataset(train_filenames, n_readers=5, n_fields=9, n_threads=5, epochs=None, batch_size=batch_size, shuffle=True)
valid_dataset = csv_reader_dataset(valid_filenames, n_readers=5, n_fields=9, n_threads=5, epochs=1, batch_size=32, shuffle=False)
test_dataset = csv_reader_dataset(test_filenames, n_readers=5, n_fields=9, n_threads=5, epochs=1, batch_size=32, shuffle=False)

## 4. 使用dataset 训练、测试

In [10]:
model = keras.models.Sequential([
    keras.layers.Dense(100, input_shape=[8], activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="sgd")

In [11]:
model.fit(
    train_dataset, 
    validation_data=valid_dataset, 
    steps_per_epoch=len(x_train)//batch_size, 
    validation_steps=len(x_valid)//batch_size, 
    epochs=epochs
)

Train for 362 steps, validate for 120 steps
Epoch 1/100
362/362 [==============================] - 1s 4ms/step - loss: 0.7297 - val_loss: 0.6342
Epoch 2/100
362/362 [==============================] - 1s 2ms/step - loss: 0.8922 - val_loss: 0.5401
Epoch 3/100
362/362 [==============================] - 1s 2ms/step - loss: 0.4830 - val_loss: 0.5984
Epoch 4/100
362/362 [==============================] - 1s 2ms/step - loss: 0.5133 - val_loss: 0.6793
Epoch 5/100
362/362 [==============================] - 1s 2ms/step - loss: 0.5260 - val_loss: 0.4166
Epoch 6/100
362/362 [==============================] - 1s 2ms/step - loss: 0.3897 - val_loss: 0.4076
Epoch 7/100
362/362 [==============================] - 1s 2ms/step - loss: 0.3997 - val_loss: 0.3986
Epoch 8/100
362/362 [==============================] - 1s 2ms/step - loss: 0.3649 - val_loss: 0.3676
Epoch 9/100
362/362 [==============================] - 1s 2ms/step - loss: 0.3557 - val_loss: 0.3655
Epoch 10/100
362/362 [=========================

Epoch 81/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2497 - val_loss: 0.3047
Epoch 82/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2589 - val_loss: 0.2918
Epoch 83/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2470 - val_loss: 0.2884
Epoch 84/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2506 - val_loss: 0.2852
Epoch 85/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2545 - val_loss: 0.2811
Epoch 86/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2441 - val_loss: 0.2765
Epoch 87/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2528 - val_loss: 0.2800
Epoch 88/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2521 - val_loss: 0.2790
Epoch 89/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2480 - val_loss: 0.2817
Epoch 90/100
362/362 [==============================] - 1s 2ms/step - loss: 0.2449

In [14]:
model.evaluate(test_dataset)

    162/Unknown - 0s 2ms/step - loss: 0.2858

0.28575366818242603